<a href="https://colab.research.google.com/github/Davron030901/PyTorch/blob/main/9_CNN_Visualisation_Filter_and_Filter_Activation_Visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Visualising What CNNs Learn**

---


In this lesson, we use **Keras with a TensorFlow 2.0** to visualise the following (see below). This helps you gain a better understanding of what's going on under the hood and de-mystifies some of the deep learning aspects.**
1. Training a basic CNN on the MNIST Dataset
2. Visualise its filters
3. Visualise the filter activations as we propagate an input image

# **Training a Basic CNN on the MNIST Dataset**

In [ ]:
# We can load the built in datasets from this function
from tensorflow.keras.datasets import mnist

# loads the MNIST training and test dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Check to see if we're using the GPU
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

# Display the number of samples in x_train, x_test, y_train, y_test
print("Initial shape or dimensions of x_train", str(x_train.shape))

# Print the number of samples in our data
print ("Number of samples in our training data: " + str(len(x_train)))
print ("Number of labels in our training data: " + str(len(y_train)))
print ("Number of samples in our test data: " + str(len(x_test)))
print ("Number of labels in our test data: " + str(len(y_test)))

# Print the image dimensions and no. of labels in our Training and Test Data
print("\n")
print ("Dimensions of x_train:" + str(x_train[0].shape))
print ("Labels in x_train:" + str(y_train.shape))
print("\n")
print ("Dimensions of x_test:" + str(x_test[0].shape))
print ("Labels in y_test:" + str(y_test.shape))

In [ ]:
# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

# Getting our data in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our data thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32') #uint8 originally
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255.0
x_test /= 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

from tensorflow.keras.utils import to_categorical

# Now we one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Let's count the number columns in our hot encoded matrix
print ("Number of Classes: " + str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(0.001),
              metrics = ['accuracy'])

print(model.summary())

In [ ]:
batch_size = 128
epochs = 10

# Store our results here so we can plot later
# In our fit function we specify our datsets (x_train & y_train),
# the batch size (typically 16 to 128 depending on your RAM), the number of
# epochs (usually 10 to 100) and our validation datasets (x_test & y_test)
# verbose = 1, sets our training to output performance metrics every epoch
history = model.fit(x_train,
                    y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

# We obtain our accuracy score using the evalute function
# Score holds two values, our Test loss and Accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 138s 293ms/step - accuracy: 0.3009 - loss: 2.2521 - val_accuracy: 0.6776 - val_loss: 2.0211
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 136s 282ms/step - accuracy: 0.7127 - loss: 1.8030 - val_accuracy: 0.8221 - val_loss: 0.8891
Epoch 3/10
459/469 ━━━━━━━━━━━━━━━━━━━━ 2s 274ms/step - accuracy: 0.8344 - loss: 0.7506

KeyboardInterrupt: 

## **Get the symbolic outputs of each "key" layer (we gave them unique names).**

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

## **Get the shapes of only our Conv Filters**

In [ ]:
# summarize filter shapes
for layer in model.layers:
  # check for convolutional layer

  if 'conv' not in layer.name:
    continue

  # get filter weights
  filters, biases = layer.get_weights()
  print(layer.name, filters.shape)

## **Let's take a look at the weights of our first Conv Layer**

In [ ]:
# retrieve weights from the first Conv (hidden) layer
filters, biases = model.layers[0].get_weights()

In [ ]:
# Let's take a look at our filters
print(filters.shape)
filters

In [ ]:
# And now let's look at our biases
print(biases.shape)
biases

## **Let's normalize filter values to 0-1 so we can visualize them**

Our Filter weights range

In [ ]:
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
print(f'Before Normalisation, Min = {f_min} and Max =  {f_max}')
filters = (filters - f_min) / (f_max - f_min)
print(f'After Normalisation, Min = {filters.min()} and Max =  {filters.max()}')

## **Visualize our Trained Fillters**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot first few filters and set plot size
n_filters, ix = 32, 1
plt.figure(figsize=(12,20))

for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    #print(f.shape)

    # Arrange in subplot of 4 x 8
    ax = plt.subplot(n_filters, 4, ix)
    ax.set_xticks([])
    ax.set_yticks([])

    # plot filter channel in grayscale
    plt.imshow(np.squeeze(f, axis=2), cmap='gray')
    ix += 1

# show the figure
plt.show()

# **Filter Activations**

In [ ]:
from tensorflow.keras.models import Model

# If you're using a Sequential model
model = Sequential([
    # Define layers with input_shape in the first layer
    Dense(64, activation='relu', input_shape=(input_dimensions,)),
    # Other layers...
])

# Alternatively, if using the Functional API
inputs = Input(shape=(input_dimensions,))
x = Dense(64, activation='relu')(inputs)
# Other layers...
model = Model(inputs=inputs, outputs=...)

# Compile the model before extraction
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Now create the activation model
layer_outputs = [layer.output for layer in model.layers[:2]]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
# If using a Sequential model
model.build((None, input_dimensions))

# Or call the model with some dummy input
dummy_input = np.zeros((1, input_dimensions))
model.predict(dummy_input)

# Then create the activation model
layer_outputs = [layer.output for layer in model.layers[:2]]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
from tensorflow.keras.models import Model

# Extracts the outputs of the top 2 layers
layer_outputs = [layer.output for layer in model.layers[:2]]

# Creates a model that will return these outputs, given the model input
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
model.input

In [ ]:
layer_outputs

In [ ]:
import matplotlib.pyplot as plt

img_tensor = x_test[22].reshape(1,28,28,1)
fig = plt.figure(figsize=(5,5))
plt.imshow(img_tensor[0,:,:,0],cmap="gray")
plt.axis('off')

## **Get the output after the 2nd Conv Filter (after ReLU)**

In [ ]:
# Returns a list of two Numpy arrays: one array per layer activation
activations = activation_model.predict(img_tensor)

print("Number of layer activations: " + str(len(activations)))

#### **The activation of the first convolution layer for the image input**

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
second_layer_activation = activations[1]
print(second_layer_activation.shape)

In [ ]:
print(model.summary())

## **The output of the 4th Conv filter Feature Map in the first Conv layer**

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 3], cmap='viridis')
plt.legend()

## **Create a function that displys the activations of specific layers**

In [ ]:
def display_activation(activations, col_size, row_size, act_index):
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='viridis')
            activation_index += 1

In [ ]:
display_activation(activations, 4, 8, 0)